<a href="https://colab.research.google.com/github/lucevito/image/blob/main/class_weight_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [2]:
import os
import uuid
import glob
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

canali_selezionati = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras import metrics
import matplotlib.pyplot as plt
from keras.utils import plot_model

train_images_path = 'Immagini_satellitari/Train/images'
train_masks_path = 'Immagini_satellitari/Train/masks'
train_images_files = glob.glob(train_images_path + '/*.npy')
train_masks_files = glob.glob(train_masks_path + '/*.npy')
test_images_path = 'Immagini_satellitari/Test/images'
test_masks_path = 'Immagini_satellitari/Test/masks'
test_images_files = glob.glob(test_images_path + '/*.npy')
test_masks_files = glob.glob(test_masks_path + '/*.npy')

train_images_batch = np.array([np.load(file) for file in train_images_files])
train_masks_batch = np.array([np.load(file) for file in train_masks_files])
train_images_batch = train_images_batch[:, :, :, canali_selezionati]


test_images = np.array([np.load(file) for file in test_images_files])
test_masks = np.array([np.load(file) for file in test_masks_files])
test_images = test_images[:, :, :, canali_selezionati]




In [4]:
train_images, val_images, train_masks, val_masks = train_test_split(train_images_batch,
                                                                    train_masks_batch,
                                                                    test_size=0.2, random_state=42)

print('train_images :')
num_pixel_per_image = 32 * 32 * 10
train_images = train_images.reshape(-1, num_pixel_per_image)
print(train_images.shape)

print('train_masks :')
num_pixel_per_mask = 32 * 32 * 1
train_masks = train_masks[:, :32, :32, np.newaxis]
train_masks = train_masks.reshape(-1, num_pixel_per_mask)
print(train_masks.shape)

print('val_images :')
num_pixel_per_image = 32 * 32 * 10
val_images = val_images.reshape(-1, num_pixel_per_image)
print(val_images.shape)

print('val_masks :')
num_pixel_per_mask = 32 * 32 * 1
val_masks = val_masks[:, :32, :32, np.newaxis]
val_masks = val_masks.reshape(-1, num_pixel_per_mask)
print(val_masks.shape)


print('test_images :')
num_pixel_per_image = 32 * 32 * 10
test_images = test_images.reshape(-1, num_pixel_per_image)
print(test_images.shape)

print('test_masks :')
num_pixel_per_mask = 32 * 32 * 1
test_masks = test_masks[:, :32, :32, np.newaxis]
test_masks = test_masks.reshape(-1, num_pixel_per_mask)
print(test_masks.shape)

train_images :
(523, 10240)
train_masks :
(523, 1024)
val_images :
(131, 10240)
val_masks :
(131, 1024)
test_images :
(161, 10240)
test_masks :
(161, 1024)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=100,class_weight = "balanced",random_state=42)
rf_model.fit(train_images, train_masks)

val_predictions = rf_model.predict(val_images)
val_predictions = (val_predictions > 0.5).astype(np.uint8)

val_accuracy = accuracy_score(val_masks, val_predictions)
print("Validation Accuracy:", val_accuracy)

test_predictions = rf_model.predict(test_images)
test_predictions = (test_predictions > 0.5).astype(np.uint8)

test_accuracy = accuracy_score(test_masks, test_predictions)
print("Test Accuracy:", test_accuracy)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: ignored

In [ ]:
print(test_masks.shape)
print(test_predictions.shape)

In [ ]:
print(test_masks)


In [ ]:
print(test_predictions)

In [ ]:
test_masks = test_masks.flatten()
test_predictions = test_predictions.flatten()
print(test_masks.shape)
print(test_predictions.shape)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Calcola l'accuracy
accuracy = accuracy_score(test_masks, test_predictions)
print("Accuracy:", accuracy)

# Calcola la precision
precision = precision_score(test_masks, test_predictions)
print("Precision:", precision)

# Calcola il recall
recall = recall_score(test_masks, test_predictions)
print("Recall:", recall)

# Calcola l'F1-score
f1 = f1_score(test_masks, test_predictions)
print("F1-score:", f1)

# Calcola la confusion matrix
conf_matrix = confusion_matrix(test_masks, test_predictions)
print("Confusion Matrix:")
print(conf_matrix)
